In [19]:
import numpy as np
global a,b

# Initialize the hyperplane parameters a and b to 0
a = np.zeros((6,1))
b = 0.0

# Load data array
x = np.load('nparray.npy')

In [20]:
def gamma_i(x_i,a_vec,b_sca):
    '''Function accepts a feature vector x_i, a and b and returns the value of a linear equation used for class prediction'''
    return np.dot(np.transpose(a_vec),x_i) + b_sca

In [24]:
'''This code snippet plots the accuracy values of a SVM and magnitude of coefficient vector 'a' (of linear classifier) against a 
set of values for the regularization constant lambda. The SVM is trained using stochastic gradient descent that optimizes a 
hinge loss function. The training proceeds in 50 epochs of 300 steps each. A 10% validation set is used to calculate accuracies
and a 80% train set is used for training. The remaining 10% test set is not used in this exercise.'''
np.random.shuffle(x)
part_index = int(.8 * x.shape[0])
train = x[:part_index,:] # 80% data for training
testnvalidation = x[part_index:,:]
part_index = int(.5 * testnvalidation.shape[0])
test = testnvalidation[:part_index,:] # 10% data for test
validate = testnvalidation[part_index:,:] # 10% data for validation 
validate_x = validate[:,0:6]
validate_y = validate[:,6]

# m and n are used to regulate the step size as epochs change and the training progresses
# k = number of steps
m,n,k = 1,50,300
batch_size = 1
acc_per_lambda_matrix = []
a_mag_per_lambda_matrix = []


for lambda_ in [1e-5,1e-4,1e-3,1e-2,1e-1,1]:
    print('lambda is',lambda_)
    a = np.zeros((6,1))
    b = 0.0
    # stores SVM accuracy values for the current lambda value 
    accuracy_list = []
    #stores magnitude of a for the current lambda value
    a_mag_list = []
    
    for epoch in range(1,51):
        # reduce step size as a function of epoch number and constants m and n
        eta = m/(0.01*epoch + n)
    
        for step in range(0,k):
            # for each step, randomly select a record from the training data
            train_k = train[np.random.choice(train.shape[0], batch_size, replace=True)]
            # separate out the train vector into features and class
            x_k, y_k = train_k[:,0:6].reshape(6,1), train_k[:,6]
            # compute revised values of a and b
            if y_k * gamma_i(x_k,a,b) >= 1:
                a_new = a - eta*lambda_*a
                a = a_new
            else:
                a_new = a - eta*(lambda_*a - y_k*x_k)
                b_new = b + eta*y_k
                a, b = a_new, b_new
            # calculate and store the accuracy and vector a magnitude every 30 steps
            if step % 30==0:
                predicted_y = np.array([gamma_i(x,a,b) for x in validate_x])
                accuracy_arr = (predicted_y * validate_y.reshape(validate_y.shape[0],1)) > 0
                accuracy_list.append(np.sum(accuracy_arr) / accuracy_arr.size)
                a_mag_list.append(np.sqrt(np.transpose(a).dot(a))[0][0])
    acc_per_lambda_matrix.append(accuracy_list)
    print('average accuracy = ',sum(accuracy_list)/len(accuracy_list))
    a_mag_per_lambda_matrix.append(a_mag_list)
print('done')

lambda is 1e-05
average accuracy =  0.809658546571
lambda is 0.0001
average accuracy =  0.809668781986
lambda is 0.001
average accuracy =  0.809296622313
lambda is 0.01
average accuracy =  0.806402865916
lambda is 0.1
average accuracy =  0.787308085977
lambda is 1
average accuracy =  0.76543828045
done


In [17]:
# create a plot of accuracy on validation set every 30 steps for each value of lambda
import matplotlib.pyplot as plt
import pandas as pd
df=pd.DataFrame(acc_per_lambda_matrix)
df=df.transpose()
f=plt.figure()
df.plot(figsize=(11,6))
plt.ylim(0.7, 0.9)
plt.xlabel('Data Points')
plt.ylabel('Accuracy')
plt.legend(['0.00001','0.0001','0.001','0.01','0.1','1'])
plt.savefig('plot1.png')

In [18]:
# create a plot of magnitude of coefficients of 'a' on validation set every 30 steps for each value of lambda
import matplotlib.pyplot as plt
import pandas as pd
mag_df = pd.DataFrame(a_mag_per_lambda_matrix)
mag_df=mag_df.transpose()
plt.figure()
mag_df.plot(figsize=(11,6))
plt.xlabel('Data Points')
plt.ylabel('Magnitude of Coefficient Vector')
plt.legend(['0.00001','0.0001','0.001','0.01','0.1','1'])
plt.savefig('plot2.png')